# Домашнее задание 1

### Выполнил Штанько Константин, группа 1, вариант 26

Для начала нам необходимо получить таблицу сопряженности по **возрасту** (меньше и больше 35 лет) и **удовлетворенности соцпакетом**:

In [40]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

df = pd.read_excel('data1.xlsx')
df = df[(df['X2'].isin([1, 2])) & (df['X13'].isin([1, 2]))]
cont_table = pd.crosstab(
    df['X2'],
    df['X13'],
    margins = True
)

cont_table.index = ['Моложе 35', 'Старше 35', 'Сумма']
cont_table.columns = ['Удовлетворен', 'Неудовлетворен', 'Сумма']

Посмотрим на таблицу сопряженности **фактических частот**:

In [57]:
cont_table

,Удовлетворен,Неудовлетворен,Сумма
Моложе 35,29,13,42
Старше 35,52,6,58
Сумма,81,19,100


#### **Пункт а):**
Построим таблицу сопряженности `теоретических частот`:

In [72]:
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(cont_table)
dof = 1

expected_table = pd.DataFrame(
    expected,
    index=cont_table.index,
    columns=cont_table.columns
)


expected_table

,Удовлетворен,Неудовлетворен,Сумма
Моложе 35,34.02,7.98,42.0
Старше 35,46.98,11.02,58.0
Сумма,81.00,19.00,100.0


#### **Пункт б):**
Получим результаты теста независимости по `информационному критерию`:

In [67]:
from scipy.stats import chi2

def g(obs, exp):
    obs = obs.values
    exp = exp.values
    G = 2 * np.sum(obs * np.log(obs / exp))
    v = (obs.shape[0] - 1) * (obs.shape[1] - 1)
    gamma = 0.975
    chi2_crit = chi2.ppf(gamma, v)
    return G, chi2_crit


G, critical = g(cont_table, expected_table)

print(f"Информационный критерий: {G:.3f}")
print(f"Критическое значение: {critical:.3f}")

Информационный критерий: 6.691
Критическое значение: 11.143


Так как критическое значение меньше наблюдаемого значения критерия, следовательно, гипотеза о независимости **отвергается** при уровне ошибки 0.025, следовательно, удовлетворенность соцпакетом и возраст связаны.

#### **Пункт в):**
Получим результаты теста независимости по `точному критерию Фишера`:

In [66]:
from scipy.stats import fisher_exact
ratio, pval = fisher_exact(cont_table.values[:-1, :-1])
print(f'Отношение шансов: {float(ratio):.3f}')
print(f'p-value: {float(pval):.3f}')

Отношение шансов: 0.257
p-value: 0.018


Так как отношение шансов меньше 1, то можно судить об обратной связи, а значение p-value (0.018) свидетельствует о том, что по критерию Фишера гипотеза о независимости **отвергается** с вероятностью ошибки 0.025, так как 0.025 > 0.018, следовательно, возраст и удовлетворенность соцпакетом связаны.

#### **Пункт г):**
Посчитаем точечные оценки `коэффициента коллигации (Y)` и `Чупрова-Крамера (CK)`:

In [102]:
chi2, p, dof, expected = chi2_contingency(cont_table.values)
n_star_star = int(np.sum(cont_table.values[:-1, :-1]))
CK = np.sqrt(chi2 / n_star_star)
var = cont_table.values[:-1, :-1]
Y = (np.sqrt(var[0][0] * var[1][1]) - np.sqrt(var[0][1] * var[1][0])) / (np.sqrt(var[0][0] * var[1][1]) + np.sqrt(var[0][1] * var[1][0]))
print(f'Коэффициент коллигации: {Y:.3f}')
print(f'Коэффициент Чупрова-Крамера: {CK:.3f}')

Коэффициент коллигации: -0.327
Коэффициент Чупрова-Крамера: 0.259


Коэффициент Чупрова-Крамера показывает слабую связь между атрибутами, где 1 - это полная связь. Коэффициент коллигации - аналогично.